In [1]:
import numpy
import cv2
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import pandas

In [2]:
IMG_SIZE = 256
sigmaX = 30

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[numpy.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][numpy.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][numpy.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = numpy.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

In [3]:
aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
dr_df = pandas.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")
print(aptos_df['diagnosis'].value_counts())
print(dr_df['level'].value_counts())

0    1805
2     999
1     370
4     295
3     193
Name: diagnosis, dtype: int64
0    25810
2     5292
1     2443
3      873
4      708
Name: level, dtype: int64


In [ ]:
import numpy
import pandas

aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
images = []
labels = []

for idx in tqdm(range(len(aptos_df))):
    images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png', 20))
    labels.append(aptos_df.iloc[idx]['diagnosis'])

from tensorflow.keras.utils import to_categorical
# images = numpy.load('../input/aptosnumpy/images.npy')
images = numpy.array(images)
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, cat_labels, 
                                                    test_size=0.1, shuffle=True,
                                                    random_state=42, stratify=labels)

  1%|▏         | 52/3662 [00:13<20:58,  2.87it/s]

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    shear_range=0.25,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     zca_whitening=True,
    zoom_range=[0.75,1.25],
    vertical_flip=True,
    horizontal_flip=True)
train_datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    shear_range=0.25,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     zca_whitening=True,
    zoom_range=[0.75,1.25],
    vertical_flip=True,
    horizontal_flip=True)
test_datagen.fit(x_test)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', numpy.unique(labels), labels)
d_class_weights = dict(enumerate(class_weights))
d_class_weights

In [5]:
import tensorflow
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D

model = Xception(include_top=False)#, drop_connect_rate=0.4)#, input_shape=(256, 256, 3))
x = GlobalMaxPooling2D()(model.output)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', 
                                                        save_best_only=True, verbose=1)
earlystopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, 
                                                         restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1,
                                                        patience=4, verbose=1)


model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=128, verbose=1,
          validation_data = (train_datagen.flow(x_train, y_train, batch_size=32)),
          callbacks=[checkpoint, earlystopping, reduceLR])#, class_weight=d_class_weights)

83689472/83683744 [==============================] - 1s 0us/step
Epoch 1/128
102/102 [==============================] - 119s 1s/step - loss: 1.1778 - accuracy: 0.5623 - val_loss: 8.5444 - val_accuracy: 0.3678

Epoch 00001: val_accuracy improved from -inf to 0.36783, saving model to model.h5
Epoch 2/128
102/102 [==============================] - 109s 1s/step - loss: 0.7357 - accuracy: 0.7297 - val_loss: 2.7139 - val_accuracy: 0.5284

Epoch 00002: val_accuracy improved from 0.36783 to 0.52838, saving model to model.h5
Epoch 3/128
102/102 [==============================] - 107s 1s/step - loss: 0.6262 - accuracy: 0.7562 - val_loss: 0.6236 - val_accuracy: 0.7678

Epoch 00003: val_accuracy improved from 0.52838 to 0.76783, saving model to model.h5
Epoch 4/128
102/102 [==============================] - 107s 1s/step - loss: 0.5828 - accuracy: 0.7630 - val_loss: 0.6360 - val_accuracy: 0.7560

Epoch 00004: val_accuracy did not improve from 0.76783
Epoch 5/128
102/102 [===========================

102/102 [==============================] - 107s 1s/step - loss: 0.2911 - accuracy: 0.8711 - val_loss: 0.2392 - val_accuracy: 0.9105

Epoch 00037: val_accuracy improved from 0.90106 to 0.91047, saving model to model.h5
Epoch 38/128
102/102 [==============================] - 107s 1s/step - loss: 0.2913 - accuracy: 0.8903 - val_loss: 0.2455 - val_accuracy: 0.9071

Epoch 00038: val_accuracy did not improve from 0.91047
Epoch 39/128
102/102 [==============================] - 108s 1s/step - loss: 0.2792 - accuracy: 0.8939 - val_loss: 0.2332 - val_accuracy: 0.9135

Epoch 00039: val_accuracy improved from 0.91047 to 0.91351, saving model to model.h5
Epoch 40/128
102/102 [==============================] - 107s 1s/step - loss: 0.2668 - accuracy: 0.8971 - val_loss: 0.2389 - val_accuracy: 0.9093

Epoch 00040: val_accuracy did not improve from 0.91351
Epoch 41/128
102/102 [==============================] - 107s 1s/step - loss: 0.2577 - accuracy: 0.8992 - val_loss: 0.2325 - val_accuracy: 0.9165

Epo